In [2]:
from PIL import Image
import pyproj

def twd97_to_pixel_coordinates(input_path, twd97_coordinates):
    # Open the TIFF image to get its projection information
    image = Image.open(input_path)
    width, height = image.size

    # Define the TWD97 and pixel coordinate systems
    twd97 = pyproj.Proj(init='epsg:3826')  # TWD97 projection EPSG code
    pixel = pyproj.Proj(init='epsg:3857')  # Web Mercator projection EPSG code

    # Convert each TWD97 coordinate to pixel coordinates
    pixel_coordinates = []
    for coord in twd97_coordinates:
        x, y = pyproj.transform(twd97, pixel, coord[0], coord[1])
        pixel_x = int(x * width / 2 + width / 2)
        pixel_y = int(-y * height / 2 + height / 2)
        pixel_coordinates.append((pixel_x, pixel_y))

    return pixel_coordinates

def clip_tiff(input_path, output_path, twd97_coordinates):
    # Convert TWD97 coordinates to pixel coordinates
    pixel_coordinates = twd97_to_pixel_coordinates(input_path, twd97_coordinates)

    # Open the TIFF image
    image = Image.open(input_path)

    # Determine the bounding box of the clipped region
    left = min(pixel_coordinates[0][0], pixel_coordinates[1][0])
    top = min(pixel_coordinates[0][1], pixel_coordinates[3][1])
    right = max(pixel_coordinates[1][0], pixel_coordinates[2][0])
    bottom = max(pixel_coordinates[2][1], pixel_coordinates[3][1])

    # Clip the image using the bounding box
    clipped_image = image.crop((left, top, right, bottom))

    # Save the clipped image
    clipped_image.save(output_path)

    print("Image clipped and saved successfully!")

# Example usage
input_file = "g:\\2024_NDHU research assistant\\DNDF\\202309_大農大富RGB&NDVI\\pix4d file\\006\\006_transparent_mosaic_group1.tif"
output_file = "g:\\2024_NDHU research assistant\\DNDF\\86_1_clipped_image.tiff"
# Example TWD97 coordinates defining the four corners of the clipping area
twd97_coordinates = [
    (292427.808, 2613025.95),
    (292426.1589, 2613025.723),
    (292326.5081, 2613041.044),
    (292328.6692, 2613020.622)

]
clip_tiff(input_file, output_file, twd97_coordinates)


DecompressionBombError: Image size (256566915 pixels) exceeds limit of 178956970 pixels, could be decompression bomb DOS attack.

In [5]:
import rasterio
from rasterio.enums import Resampling
from rasterio.transform import from_origin
from rasterio.warp import reproject, Resampling

def twd97_to_pixel_coordinates(input_path, twd97_coordinates):
    # Open the TIFF image to get its projection information
    with rasterio.open(input_path) as src:
        left, top = src.index(twd97_coordinates[0][0], twd97_coordinates[0][1])
        right, bottom = src.index(twd97_coordinates[2][0], twd97_coordinates[2][1])
    return left, top, right, bottom

def clip_tiff(input_path, output_path, twd97_coordinates):
    # Convert TWD97 coordinates to pixel coordinates
    left, top, right, bottom = twd97_to_pixel_coordinates(input_path, twd97_coordinates)
    print(left, top, right, bottom)

    # Open the TIFF image
    with rasterio.open(input_path) as src:
        # Define the window to read
        window = ((top, bottom), (left, right))

        # Read the data within the window
        clipped_data = src.read(window=window)

        # Update the transform to reflect the new window
        transform = from_origin(src.bounds.left + left * src.res[0], src.bounds.top - top * src.res[1], src.res[0], src.res[1])

        # Update the metadata
        kwargs = src.meta.copy()
        kwargs.update({
            'height': bottom - top,
            'width': right - left,
            'transform': transform
        })

        # Write the clipped data to a new file
        with rasterio.open(output_path, 'w', **kwargs) as dst:
            dst.write(clipped_data)

    print("Image clipped and saved successfully!")

# Example usage
input_file = "g:\\2024_NDHU research assistant\\DNDF\\202309_大農大富RGB&NDVI\\pix4d file\\006\\006_transparent_mosaic_group1.tif"
output_file = "g:\\2024_NDHU research assistant\\DNDF\\86_1_clipped_image.tiff"
# Example TWD97 coordinates defining the four corners of the clipping area
twd97_coordinates = [
    (292427.808, 2613025.95),
    (292426.1589, 2613025.723),
    (292326.5081, 2613041.044),
    (292328.6692, 2613020.622)
]
clip_tiff(input_file, output_file, twd97_coordinates)


12887 11059 12595 9097


RasterioIOError: Attempt to create -292x-1962 dataset is illegal,sizes must be larger than zero.

In [21]:
import rasterio
from rasterio.enums import Resampling
from rasterio.transform import from_origin
from rasterio.warp import reproject, Resampling
from affine import Affine

def twd97_to_pixel_coordinates(input_path, twd97_coordinates):
    # Open the TIFF image to get its projection information
    with rasterio.open(input_path) as src:
        pixel_coordinates = [src.index(coord[0], coord[1]) for coord in twd97_coordinates]
    return pixel_coordinates

def clip_tiff(input_path, output_path, twd97_coordinates):
    # Convert TWD97 coordinates to pixel coordinates
    pixel_coordinates = twd97_to_pixel_coordinates(input_path, twd97_coordinates)
    print(pixel_coordinates)

    # Extract coordinates
    bottom_right, top_right, top_left, bottom_left = pixel_coordinates

    # Open the TIFF image
    with rasterio.open(input_path) as src:
        # Define the window to read
        window = ((top_left[0], bottom_left[0]+1), (top_left[1], top_right[1]+1))

        # Read the data within the window
        clipped_data = src.read(window=window)

        # Update the transform to reflect the new window
        transform = src.transform
        transform = Affine(transform.a, transform.b, 292326.5081, transform.d, transform.e, 2613041.044)

        # Update the metadata
        print(src.meta)
        kwargs = src.meta.copy()
        kwargs.update({
            'height': bottom_left[0] - top_left[0],
            'width': top_right[1] - top_left[1],
            'transform': transform
        })

        print(kwargs)

        # Write the clipped data to a new file
        with rasterio.open(output_path, 'w', **kwargs) as dst:
            dst.write(clipped_data)

    print("Image clipped and saved successfully!")

# Example usage
input_file = "g:\\2024_NDHU research assistant\\DNDF\\202309_大農大富RGB&NDVI\\pix4d file\\006\\006_transparent_mosaic_group1.tif"
output_file = "g:\\2024_NDHU research assistant\\DNDF\\86_1_clipped_image_2.tiff"
# Example TWD97 coordinates defining the four corners of the clipping area
twd97_coordinates = [
    (292427.808, 2613025.95),
    (292426.1589, 2613025.723),
    (292326.5081, 2613041.044),
    (292328.6692, 2613020.622)
]
clip_tiff(input_file, output_file, twd97_coordinates)


[(12887, 11059), (12891, 11027), (12595, 9097), (12990, 9139)]
{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 16115, 'height': 15921, 'count': 4, 'crs': CRS.from_epsg(3826), 'transform': Affine(0.051620000000000006, 0.0, 291856.90213,
       0.0, -0.051620000000000006, 2613691.20543)}
{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 1930, 'height': 395, 'count': 4, 'crs': CRS.from_epsg(3826), 'transform': Affine(0.051620000000000006, 0.0, 292326.5081,
       0.0, -0.051620000000000006, 2613041.044)}
Image clipped and saved successfully!
